In [1]:
import zipfile
import os
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('all')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, classification_report, recall_score
import warnings
from pandas.errors import SettingWithCopyWarning
import joblib
import tensorflow as tf
import os
import torch
from transformers import BertTokenizer, BertModel
import umap
import hdbscan
from sklearn.linear_model import LogisticRegression
from bertopic import BERTopic

uploaded_zip = '/home/abarovic/popeREU/eclipse_platform.zip'
extract_dir = '/home/abarovic/popeREU'
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)
with zipfile.ZipFile(uploaded_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

csv_file = os.path.join(extract_dir, 'eclipse_platform.csv')
fulldata = pd.read_csv(csv_file)

print(fulldata.columns)
print(fulldata.isna().sum())  # Check NaNs in training features

def CreateDuration(dataset):
    dataset['Created_time'] = pd.to_datetime(dataset['Created_time'], utc=True)
    dataset['Resolved_time'] = pd.to_datetime(dataset['Resolved_time'], utc=True)

    # Convert to the desired format
    dataset['Created_time2'] = dataset['Created_time'].dt.strftime('%m/%d/%Y %H:%M')
    dataset['Resolved_time2'] = dataset['Resolved_time'].dt.strftime('%m/%d/%Y %H:%M')

    # Parse the formatted datetime strings back to datetime objects to ensure they are in the correct format
    dataset['Created_time2'] = pd.to_datetime(dataset['Created_time2'], format='%m/%d/%Y %H:%M')
    dataset['Resolved_time2'] = pd.to_datetime(dataset['Resolved_time2'], format='%m/%d/%Y %H:%M')

    # Calculate the duration in hours
    dataset['Duration'] = dataset['Resolved_time2'] - dataset['Created_time2']
    dataset['Duration_hours'] = dataset['Duration'].dt.total_seconds() / 3600

    # Drop the intermediate columns
    dataset.drop(['Created_time2', 'Resolved_time2', 'Duration'], axis=1, inplace=True)

CreateDuration(fulldata)

def NoOutliers(df):
    Q1 = df['Duration_hours'].quantile(0.25)
    Q3 = df['Duration_hours'].quantile(0.75)

    # Calculate the IQR
    IQR = Q3 - Q1

    # Define the lower and upper bounds for filtering outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter out the outliers
    df = df[(df['Duration_hours'] >= lower_bound) & (df['Duration_hours'] <= upper_bound)]
    return(df)

#fulldata = NoOutliers(fulldata)

def CreateTimeLabel(df):
    threshold = df['Duration_hours'].quantile(0.70)
    # Create a new column TimeLabel based on the threshold
    df['TimeLabel'] = df['Duration_hours'].apply(lambda x: 'short' if x <= threshold else 'long')
    df = df[df['Resolution'] != 'MOVED']
    #df = df[df['Resolution'] == 'FIXED']
    return df

fulldata = CreateTimeLabel(fulldata)
def RemoveStopWords(dataset):
    #making all the items in the descriptions columns lower
    dataset['Description'] = dataset['Description'].str.lower()
    # Define stop words
    stop_words = set(stopwords.words('english'))
     # Remove stop words
    for index, row in dataset.iterrows():
        # Check if the 'Description' is not NaN
        if isinstance(row['Description'], str):
            words = row['Description'].split()
            final_tokens = [word for word in words if word not in stop_words]
            dataset.at[index, 'Description'] = ' '.join(final_tokens)
    return dataset

fulldata = RemoveStopWords(fulldata)

def Lemmitization(dataset):
    # Initialize the WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Function to lemmatize a sentence
    def lemmatize_sentence(sentence):
        words = nltk.word_tokenize(sentence)
        lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in words]  # Lemmatize verbs (default)
        return ' '.join(lemmatized_words)
    dataset['Description'] = dataset['Description'].astype(str)  # Convert all to strings
    # Apply lemmatization to each row in the 'Description' column
    dataset['Description'] = dataset['Description'].apply(lemmatize_sentence)

Lemmitization(fulldata)

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return None

# Function to calculate sentiment scores
def CalculateSentimentScores(description):
    tokens = word_tokenize(description)
    tagged_tokens = pos_tag(tokens)
        
    pos_score = 0
    neg_score = 0
    token_count = 0
    for word, tag in tagged_tokens:
        wn_tag = get_wordnet_pos(tag)
        if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
            continue
        
        synsets = list(swn.senti_synsets(word, wn_tag))
        if not synsets:
            continue
        
        # Use the first synset for simplicity
        synset = synsets[0]
        pos_score += synset.pos_score()
        neg_score += synset.neg_score()
        token_count += 1
    
    # Normalize scores by the number of tokens
    if token_count > 0:
        pos_score /= token_count
        neg_score /= token_count
        return pos_score, neg_score

# Apply sentiment score calculation to each description
def CreatePosNegColumns(dataset): 
    dataset[['Pos_Score', 'Neg_Score']] = dataset['Description'].apply(lambda x: pd.Series(CalculateSentimentScores(x)))

CreatePosNegColumns(fulldata)

def EmotionColumn(dataset):
    #Creating a column for binary emotion (positive or negative)
    dataset['Emotion'] = dataset['Pos_Score'] - dataset['Neg_Score']
    
    # Assign labels based on the difference
    dataset['Emotion'] = dataset['Emotion'].apply(lambda x: 'positive' if x > 0 else 'negative')

EmotionColumn(fulldata)

def EmotionalityColumn(dataset):
    dataset['Emotionality'] = dataset['Pos_Score']+ dataset['Neg_Score']

EmotionalityColumn(fulldata)

def Destiny(df):
    # Create a new column 'Destiny' with default value 'Not Fixed'
    df['Destiny'] = 'Not Fixed'
    # Update 'Destiny' based on conditions in 'Resolution' column
    df.loc[df['Resolution'] == 'FIXED', 'Destiny'] = 'Fixed'
    return df

fulldata = Destiny(fulldata)
#print(fulldata['Emotion'].head(100))

def Emotion(df):
    # Fill NaN values with a default value (e.g., 'neutral') before mapping
    df['Emotion'] = df['Emotion'].fillna('neutral')
    # Label Positive emotion as 1, negative as 0, and neutral as -1
    df['Emotion'] = df['Emotion'].map({'positive': 1, 'negative': 0, 'neutral': -1})
    return df

fulldata = Emotion(fulldata)


def Priority(df):
    # Fill NaN values with a default value (e.g., 'P5') before mapping
    df['Priority'] = df['Priority'].fillna('P5')
    # Label Priority Columns numerically
    df['Priority'] = df['Priority'].map({'P1': 1, 'P2': 2, 'P3': 3, 'P4': 4, 'P5': 5})
    return df

fulldata = Priority(fulldata)

def DropDescriptionNA(df):
    df = df[df['Description'] != 'nan']

DropDescriptionNA(fulldata)

def Model(df):
    model = BERTopic(nr_topics=20)

    # Prepare the documents
    docs = df['Description'].fillna('').astype(str).tolist()

    # Fit the model to your data
    topics, probabilities = model.fit_transform(docs)
    #adding a column for predicted topics
    new_docs = df['Description'].fillna('').astype(str).to_list()
    topics, probs = model.transform(new_docs)
    df['Predicted_Topic'] = topics

    # Get topic information
    topic_info = model.get_topic_info()
    print(topic_info)
    # Get words associated with each topic
    topic_words = {topic: model.get_topic(topic) for topic in topic_info.Topic}
    print(topic_words)
    return df

fulldata = Model(fulldata)

print(fulldata.columns)

print(fulldata.columns)
print(fulldata['Emotionality'].head(100))

def SplitData(dataset):
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    trainset, testset = dataset[:train_size], dataset[train_size:]
    return trainset, testset
trainset, testset = SplitData(fulldata)
print("Total items in dataset:", fulldata.shape[0])
print("Total items in trainset:", trainset.shape[0])
print("Total items in testset:", testset.shape[0])

print(trainset.columns)
print(testset.columns)

print(trainset['Emotionality'].head(100))

print("-----------------\n Training Set: \n-----------------")
print(trainset.isna().sum())  # Check NaNs in training features
print("-----------------\n Test Set: \n-----------------")
print(testset.isna().sum())   # Check NaNs in test features


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /home/abarovic/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]  

Index(['Issue_id', 'Priority', 'Component', 'Duplicated_issue', 'Title',
       'Description', 'Status', 'Resolution', 'Version', 'Created_time',
       'Resolved_time'],
      dtype='object')
Issue_id                0
Priority                0
Component               0
Duplicated_issue    70752
Title                   0
Description           129
Status                  0
Resolution              0
Version                 0
Created_time            0
Resolved_time           0
dtype: int64


/home/abarovic/anaconda3/envs/pope_env/lib/python3.8/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The curren

    Topic  Count                                             Name  \
0      -1  42410                         -1_file_eclipse_use_line   
1       0  27286                        0_at_00000000_file_native   
2       1  10715                       1_new_shell_display_public   
3       2   1453                        2_return_object_new_class   
4       3    740                            3_note_dsgsg_w00t_den   
5       4    599                     4_search_history_view_result   
6       5    507                         5_bug_clone_backport_fix   
7       6    379                    6_feature_update_install_site   
8       7    345                     7_print_console_0303_printer   
9       8    307                 8_sort_order_column_viewersorter   
10      9    130                                    9_nan_bogus__   
11     10     72  10_gtkwarning_gtkwidgetsizeallocate_allocate_f1   
12     11     45                             11_none_seq_rel_note   
13     12     34                 1

CNN (Emotion, Emotionality, priority)

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

trainset, testset = SplitData(fulldata)

# Filter only rows where Destiny is 'Fixed'
trainset = trainset[trainset['Destiny'] == 'Fixed']
testset = testset[testset['Destiny'] == 'Fixed']

# Define features and target
features = ['Emotion', 'Emotionality', 'Priority']
target = 'TimeLabel'

# Ensure 'TimeLabel' is mapped to binary values
trainset['TimeLabel'] = trainset['TimeLabel'].map({'short': 0, 'long': 1})
testset['TimeLabel'] = testset['TimeLabel'].map({'short': 0, 'long': 1})

# Handle missing values (Fill NaN with median value)
imputer = SimpleImputer(strategy='median')  # You can also use 'mean' or 'most_frequent'

# Apply imputer to features
X_train = imputer.fit_transform(trainset[features])
X_test = imputer.transform(testset[features])

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Target variable
y_train = trainset[target].dropna()  # Ensure no NaNs in target
y_test = testset[target].dropna()

# Define CNN Model
model = Sequential([
    Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification (short vs long)
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Epoch 1/20


2025-03-06 18:48:29.087705: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


2040/2040 [==============================] - 7s 3ms/step - loss: 0.5528 - accuracy: 0.7609 - val_loss: 0.5170 - val_accuracy: 0.7936
Epoch 2/20
2040/2040 [==============================] - 6s 3ms/step - loss: 0.5475 - accuracy: 0.7637 - val_loss: 0.5129 - val_accuracy: 0.7935
Epoch 3/20
2040/2040 [==============================] - 6s 3ms/step - loss: 0.5465 - accuracy: 0.7643 - val_loss: 0.5116 - val_accuracy: 0.7938
Epoch 4/20
2040/2040 [==============================] - 6s 3ms/step - loss: 0.5463 - accuracy: 0.7641 - val_loss: 0.5111 - val_accuracy: 0.7938
Epoch 5/20
2040/2040 [==============================] - 6s 3ms/step - loss: 0.5458 - accuracy: 0.7641 - val_loss: 0.5105 - val_accuracy: 0.7936
Epoch 6/20
2040/2040 [==============================] - 6s 3ms/step - loss: 0.5453 - accuracy: 0.7643 - val_loss: 0.5097 - val_accuracy: 0.7936
Epoch 7/20
2040/2040 [==============================] - 6s 3ms/step - loss: 0.5454 - accuracy: 0.7643 - val_loss: 0.5132 - val_accuracy: 0.7938
Epo

CNN (Emotion, Emotionality, priority, BERT)

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

trainset, testset = SplitData(fulldata)

# Filter only rows where Destiny is 'Fixed'
trainset = trainset[trainset['Destiny'] == 'Fixed']
testset = testset[testset['Destiny'] == 'Fixed']

# Define features and target
features = ['Emotion', 'Emotionality', 'Priority', 'Predicted_Topic']
target = 'TimeLabel'

# Ensure 'TimeLabel' is mapped to binary values
trainset['TimeLabel'] = trainset['TimeLabel'].map({'short': 0, 'long': 1})
testset['TimeLabel'] = testset['TimeLabel'].map({'short': 0, 'long': 1})

# Handle missing values (Fill NaN with median value)
imputer = SimpleImputer(strategy='median')  # You can also use 'mean' or 'most_frequent'

# Apply imputer to features
X_train = imputer.fit_transform(trainset[features])
X_test = imputer.transform(testset[features])

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Target variable
y_train = trainset[target].dropna()  # Ensure no NaNs in target
y_test = testset[target].dropna()

# Define CNN Model
model = Sequential([
    Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification (short vs long)
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Epoch 1/20
2040/2040 [==============================] - 7s 3ms/step - loss: 0.5512 - accuracy: 0.7617 - val_loss: 0.5130 - val_accuracy: 0.7938
Epoch 2/20
2040/2040 [==============================] - 6s 3ms/step - loss: 0.5468 - accuracy: 0.7639 - val_loss: 0.5128 - val_accuracy: 0.7936
Epoch 3/20
2040/2040 [==============================] - 6s 3ms/step - loss: 0.5462 - accuracy: 0.7642 - val_loss: 0.5157 - val_accuracy: 0.7938
Epoch 4/20
2040/2040 [==============================] - 6s 3ms/step - loss: 0.5460 - accuracy: 0.7642 - val_loss: 0.5097 - val_accuracy: 0.7938
Epoch 5/20
2040/2040 [==============================] - 6s 3ms/step - loss: 0.5451 - accuracy: 0.7640 - val_loss: 0.5137 - val_accuracy: 0.7938
Epoch 6/20
2040/2040 [==============================] - 6s 3ms/step - loss: 0.5455 - accuracy: 0.7642 - val_loss: 0.5098 - val_accuracy: 0.7937
Epoch 7/20
2040/2040 [==============================] - 6s 3ms/step - loss: 0.5444 - accuracy: 0.7641 - val_loss: 0.5153 - val_accuracy:

CNN (Emotion, emotionality, priority) -> Weighted

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from collections import Counter

trainset, testset = SplitData(fulldata)

# Filter only rows where Destiny is 'Fixed'
trainset = trainset[trainset['Destiny'] == 'Fixed']
testset = testset[testset['Destiny'] == 'Fixed']

# Define features and target
features = ['Emotion', 'Emotionality', 'Priority']
target = 'TimeLabel'

# Ensure 'TimeLabel' is mapped to binary values
trainset['TimeLabel'] = trainset['TimeLabel'].map({'short': 0, 'long': 1})
testset['TimeLabel'] = testset['TimeLabel'].map({'short': 0, 'long': 1})

# Handle missing values (Fill NaN with median value)
imputer = SimpleImputer(strategy='median')  # You can also use 'mean' or 'most_frequent'

# Apply imputer to features
X_train = imputer.fit_transform(trainset[features])
X_test = imputer.transform(testset[features])

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Target variable
y_train = trainset[target].dropna()  # Ensure no NaNs in target
y_test = testset[target].dropna()

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
print("Resampled class distribution:", Counter(y_train))

# Define CNN Model
model = Sequential([
    Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification (short vs long)
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Resampled class distribution: Counter({1: 24490, 0: 24490})
Epoch 1/20
3062/3062 [==============================] - 9s 3ms/step - loss: 0.6752 - accuracy: 0.5417 - val_loss: 0.6801 - val_accuracy: 0.6019
Epoch 2/20
3062/3062 [==============================] - 9s 3ms/step - loss: 0.6731 - accuracy: 0.5430 - val_loss: 0.6718 - val_accuracy: 0.5955
Epoch 3/20
3062/3062 [==============================] - 9s 3ms/step - loss: 0.6732 - accuracy: 0.5412 - val_loss: 0.6810 - val_accuracy: 0.5248
Epoch 4/20
3062/3062 [==============================] - 9s 3ms/step - loss: 0.6722 - accuracy: 0.5449 - val_loss: 0.7007 - val_accuracy: 0.4004
Epoch 5/20
3062/3062 [==============================] - 9s 3ms/step - loss: 0.6724 - accuracy: 0.5445 - val_loss: 0.6856 - val_accuracy: 0.4849
Epoch 6/20
3062/3062 [==============================] - 9s 3ms/step - loss: 0.6720 - accuracy: 0.5438 - val_loss: 0.6872 - val_accuracy: 0.5494
Epoch 7/20
3062/3062 [==============================] - 8s 3ms/step - loss: 

CNN (Emotion, emotionality, priority, BERT) -> Weighted

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from collections import Counter

trainset, testset = SplitData(fulldata)

# Filter only rows where Destiny is 'Fixed'
trainset = trainset[trainset['Destiny'] == 'Fixed']
testset = testset[testset['Destiny'] == 'Fixed']

# Define features and target
features = ['Emotion', 'Emotionality', 'Priority', 'Predicted_Topic']
target = 'TimeLabel'

# Ensure 'TimeLabel' is mapped to binary values
trainset['TimeLabel'] = trainset['TimeLabel'].map({'short': 0, 'long': 1})
testset['TimeLabel'] = testset['TimeLabel'].map({'short': 0, 'long': 1})

# Handle missing values (Fill NaN with median value)
imputer = SimpleImputer(strategy='median')  # You can also use 'mean' or 'most_frequent'

# Apply imputer to features
X_train = imputer.fit_transform(trainset[features])
X_test = imputer.transform(testset[features])

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Target variable
y_train = trainset[target].dropna()  # Ensure no NaNs in target
y_test = testset[target].dropna()

# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
print("Resampled class distribution:", Counter(y_train))


# Define CNN Model
model = Sequential([
    Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Binary classification (short vs long)
])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test))

# Evaluate model
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Resampled class distribution: Counter({1: 24490, 0: 24490})
Epoch 1/20
3062/3062 [==============================] - 10s 3ms/step - loss: 0.6746 - accuracy: 0.5450 - val_loss: 0.6680 - val_accuracy: 0.6224
Epoch 2/20
3062/3062 [==============================] - 9s 3ms/step - loss: 0.6727 - accuracy: 0.5457 - val_loss: 0.6953 - val_accuracy: 0.4067
Epoch 3/20
3062/3062 [==============================] - 9s 3ms/step - loss: 0.6721 - accuracy: 0.5517 - val_loss: 0.6784 - val_accuracy: 0.5408
Epoch 4/20
3062/3062 [==============================] - 9s 3ms/step - loss: 0.6715 - accuracy: 0.5523 - val_loss: 0.6988 - val_accuracy: 0.4331
Epoch 5/20
3062/3062 [==============================] - 8s 3ms/step - loss: 0.6713 - accuracy: 0.5509 - val_loss: 0.6664 - val_accuracy: 0.6119
Epoch 6/20
3062/3062 [==============================] - 9s 3ms/step - loss: 0.6706 - accuracy: 0.5524 - val_loss: 0.6676 - val_accuracy: 0.5529
Epoch 7/20
3062/3062 [==============================] - 9s 3ms/step - loss: